In [1]:
# breakdown yang wrong reject (WR), apakah dia jadi benar atau salah

In [2]:
# PERTANYAAN: apakah harus capital sensitive?

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from collections import OrderedDict
import pandas as pd
import re
from tqdm import tqdm
import string
import collections
import numpy as np

from io import StringIO
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [4]:
prefix = "https://huggingface.co/muhammadravi251001/fine-tuned-FilteringNLI-"
suffix = "/blob/main/results/evaluation/metric_result.txt"

suffix_df = "/raw/main/results/output/output_df.csv"
suffix_squadid_df = "/resolve/main/results/output/output_df.csv"

In [15]:
def take_dataframe(url): 
    
    response = requests.get(url)
    
    if response.status_code == 200:
        csv_data = StringIO(response.text)
        df = pd.read_csv(csv_data, index_col=0)
        df = df[['Context', 'Question', 'Prediction Answer', 'Rec. Pred Answer', 'Gold Answer', 'Properties']]
        df.fillna('', inplace=True)
    
    else:
        print("Failed to download CSV")
    
    return df

In [83]:
def find_sample_of_wr(url):
    
    df = take_dataframe(url)
    
    wr_rows = []

    for i in range(len(df)):
        
        gold_answer = df['Gold Answer'][i]
        qa_generated_answer = eval(df['Rec. Pred Answer'][i])[0] # Kandidat pertama
        nli_validated_answer = df['Prediction Answer'][i]
        
        wr = (gold_answer != qa_generated_answer) and (qa_generated_answer != nli_validated_answer)
        
        if wr:
            wr_rows.append(df.iloc[i])

    wr_df = pd.DataFrame(wr_rows).reset_index(drop=True)
    wr_df["Analysis"] = "-"

    return wr_df

# Sample WR DF

In [84]:
data = "idkmrc"
msc = f"indonli_mnli_{data}-nli"
tq = "1"
msi = "3"
var = "2"
th = "0.5"

url = f"{prefix}{msc}-{data}-TQ{tq}-TS4-MS{msi}-VA{var}-TH{th}{suffix_df}"

wr_df = pd.DataFrame(find_sample_of_wr(url))
wr_df

,Context,Question,Prediction Answer,Rec. Pred Answer,Gold Answer,Properties,Analysis
0,Protista adalah mikroorganisme eukariota yang ...,Mengapa protista tidak dikelompokkan ke dalam ...,karena bersifat parafiletik,"['', 'karena bersifat parafiletik', '']","Dari sudut pandang taksonomi, pengelompokan in...",Answer saved from variation 2 with founded index,-
1,"Liu Ju (Chinese:劉據; 128-91 SM), secara resmi d...",Kapan Liu Ju meninggal?,"128-91 SM), secara resmi dikenal sebagai Putra...","['', '128-91 SM), secara resmi dikenal sebagai...",91 SM,Answer saved from variation 2 with founded index,-
2,"Pada tahun 1884, George Eastman menemukan pita...",Kapan Kinetoskop diciptakan ?,"1884, George Eastman menemukan pita film ( sel...","['1889 dan 1892', '1884, George Eastman menemu...",1889,Answer saved from variation 2 with founded index,-
3,Penggunaan kata Halloween atau Hallowe'en bera...,Istilah Halloween apa yang sudah tidak ditemuk...,All Hallows' Eve,"['', ""All Hallows' Eve"", '']",All Hallows' Eve,Answer saved from variation 2 with founded index,-
4,"Terakhir, setelah diumumkan oleh BNPB pada 10 ...",Berapakah jumlah korban jiwa tsunami Sulawesi ...,"Terakhir, setelah diumumkan oleh BNPB pada 10 ...","['', 'Terakhir, setelah diumumkan oleh BNPB pa...",2.045,Answer saved from variation 2 with founded index,-
5,Kabupaten Takalar adalah sebuah kabupaten di p...,berapakah luas Takalar ?,"566,51km","['', '566,51km', 'wilayah 566,51']","566,51km²",Answer saved from variation 2 with founded index,-
6,Kabupaten Takalar adalah sebuah kabupaten di p...,berapakah luas Kabupaten Takalar?,"566,51km","['', '566,51km', 'wilayah 566,51']","566,51km²",Answer saved from variation 2 with founded index,-
7,"Pada tahun 1996, Vega berperan sebagai Harding...",Apakah film pertama Alexa Ellesse Vega?,Twister. Ia menjadi bintang tamu dalam banyak ...,"['', 'Twister. Ia menjadi bintang tamu dalam b...",1996,Answer saved from variation 2 with founded index,-
8,Pelemparan di Praha (English: Defenestrations ...,Dimana Pelemparan di Praha terjadi?,Prague; Czech: Pražská defenestrace) mengacu p...,"['', 'Prague; Czech: Pražská defenestrace) men...",Bohemia,Answer saved from variation 2 with founded index,-
9,Tujuan utama dari upacara Kwangkay adalah untu...,Apakah tujuan utama upacara adat Kuangkay dila...,untuk menghormati dan memuliakan roh para lelu...,"['', 'untuk menghormati dan memuliakan roh par...",menghormati dan memuliakan roh para leluhur ya...,Answer saved from variation 2 with founded index,-


In [89]:
wr_df["Analysis"][0] = "prediksi benar (terlalu singkat)"
wr_df["Analysis"][1] = "prediksi salah (terlalu panjang)"
wr_df["Analysis"][2] = "prediksi salah (terlalu panjang)"
wr_df["Analysis"][3] = "prediksi benar (berubah jadi benar)"
wr_df["Analysis"][4] = "prediksi benar (terlalu panjang)"
wr_df["Analysis"][5] = "prediksi benar (kurang satuan)"
wr_df["Analysis"][6] = "prediksi benar (kurang satuan)"
wr_df["Analysis"][7] = "salah gold answer, pred answer benar"
wr_df["Analysis"][8] = "prediksi salah (terlalu panjang)"
wr_df["Analysis"][9] = "prediksi benar (terlalu panjang)"
wr_df["Analysis"][10] = "prediksi benar (terlalu panjang)"
wr_df["Analysis"][11] = "prediksi salah (benar-benar salah)"
wr_df["Analysis"][12] = "prediksi salah (terlalu panjang)"
wr_df["Analysis"][13] = "prediksi benar (terlalu panjang)"
wr_df["Analysis"][14] = "prediksi salah (benar-benar salah)"
wr_df["Analysis"][15] = "prediksi salah (benar-benar salah)"
wr_df["Analysis"][16] = "prediksi salah (terlalu panjang)"
wr_df["Analysis"][17] = "prediksi benar (terlalu panjang)"
wr_df["Analysis"][18] = "prediksi salah (benar-benar salah)"
wr_df["Analysis"][19] = "prediksi benar (berubah jadi benar)"
wr_df["Analysis"][20] = "prediksi salah (terlalu panjang)"
wr_df["Analysis"][21] = "question tidak terlalu spesifik, pred & gold answer bisa jadi benar"
wr_df["Analysis"][22] = "prediksi benar (berubah jadi benar)"
wr_df["Analysis"][23] = "prediksi salah (benar-benar salah)"
wr_df["Analysis"][24] = "answer tidak dapat diekstrak dari context"
wr_df["Analysis"][25] = "prediksi benar (terlalu panjang)"
wr_df["Analysis"][26] = "prediksi benar (terlalu singkat)"
wr_df["Analysis"][27] = "prediksi benar (berubah jadi benar)"
wr_df["Analysis"][28] = "prediksi salah (benar-benar salah)"
wr_df["Analysis"][29] = "prediksi benar (kelebihan tutup kurung)"
wr_df["Analysis"][30] = "prediksi salah (benar-benar salah)"
wr_df["Analysis"][31] = "prediksi salah (terlalu panjang)"
wr_df["Analysis"][32] = "prediksi benar (berubah jadi benar)"
wr_df["Analysis"][33] = "prediksi salah (terlalu panjang)"
wr_df["Analysis"][34] = "prediksi benar (salah kapitalisasi saja)"
wr_df["Analysis"][35] = "prediksi benar (terlalu panjang)"

In [90]:
wr_df["Analysis"].value_counts()

prediksi salah (terlalu panjang)                                       8
prediksi benar (terlalu panjang)                                       7
prediksi salah (benar-benar salah)                                     7
prediksi benar (berubah jadi benar)                                    5
prediksi benar (terlalu singkat)                                       2
prediksi benar (kurang satuan)                                         2
salah gold answer, pred answer benar                                   1
question tidak terlalu spesifik, pred & gold answer bisa jadi benar    1
answer tidak dapat diekstrak dari context                              1
prediksi benar (kelebihan tutup kurung)                                1
prediksi benar (salah kapitalisasi saja)                               1
Name: Analysis, dtype: int64

In [91]:
wr_df.to_csv("wr_df_analysis.csv")

In [33]:
def check_wr(i, data=wr_df):
    
    print("Context:")
    print(data['Context'][i])
    print()
    
    print("Question:")
    print(data['Question'][i])
    print()
    
    print("Pred Answer:")
    print(data['Prediction Answer'][i])
    print()
    
    print("Gold Answer:")
    print(data['Gold Answer'][i])

In [81]:
check_wr(35)

Context:
Setelah Perang Dunia II, Boganda didesak oleh Uskup Bangui Mgr. Grandin untuk melengkapi karya-karya kemanusiaan dan sosialnya melalui aksi politik. Boganda memutuskan untuk ikut dalam pemilihan Majelis Nasional Perancis. Pada tanggal 10 November 1946, ia menjadi orang Oubangui pertama yang terpilih dalam majelis setelah memenangi hampir separuh dari total suara dan mengalahkan tiga kandidat lainnya, termasuk petahana yang tersisihkan, François Joseph Reste, yang sebelumnya menjabat sebagai Gubernur-Jenderal Afrika Ekuatorial Perancis.[9] Boganda tiba di Paris dengan jubah imamnya dan memperkenalkan diri kepada sesamanya para legislator sebagai putra dari seorang kanibal yang berpoligami.[10] Sejak tahun 1947 dan seterusnya, Boganda melakukan kampanye dengan penuh semangat untuk menentang rasisme dan rezim kolonial. Tidak lama setelah menyadari keterbatasan pengaruhnya di Perancis (ia bertugas di parlemen sampai tahun 1958 namun secara bertahap melepaskan diri dari aktivitas-a

# Count WR that be right and wrong answer

In [8]:
def to_right_or_wrong(url):
    
    df = find_sample_of_wr(url)
    
    to_right_ctr = 0
    to_wrong_ctr = 0
    
    for i in range(len(df)):
        
        pred_answer = df['Prediction Answer'][i]
        gold_answer = df['Gold Answer'][i]
        
        # Check for capital sensitive?
        if pred_answer == gold_answer:
            to_right_ctr += 1
        else:
            to_wrong_ctr += 1
            
    return to_right_ctr, to_wrong_ctr

In [9]:
def take_all_url_nli_validator(data, df=False):
    
    if not df:
        baseline = f"{prefix}indonli_mnli-{data}-TQ2-TS4-MS3-VA0-TH0.0{suffix}"
    else:
        if data == "idkmrc" or data == "tydiqaid":
            baseline = f"{prefix}indonli_mnli-{data}-TQ2-TS4-MS3-VA0-TH0.0{suffix_df}"
        elif data == "squadid":
            baseline = f"{prefix}indonli_mnli-{data}-TQ2-TS4-MS3-VA0-TH0.0{suffix_squadid_df}"
    
    msc_arr = ["indonli", "indonli_mnli", f"indonli_mnli_{data}-nli"]
    tq_arr = [1, 2]
    msi_arr = [1, 2, 3]
    var_arr = [1, 2, 3]
    th_arr = [0.25, 0.5, 0.75]

    url_arr = []
    url_arr.append(baseline)

    for msc in msc_arr:
        for tq in tq_arr:
            for msi in msi_arr:
                for var in var_arr:
                    for th in th_arr:
                        
                        if msc == "indonli" or msc == "indonli_mnli":
                            var = 1
                            th = 0.0

                        if var == 1:
                            th = 0.0
                            
                        if (var == 3) and (msi == 1):
                            continue
                        
                        if not df:
                            url_arr.append(f"{prefix}{msc}-{data}-TQ{tq}-TS4-MS{msi}-VA{var}-TH{th}{suffix}")
                        else:
                            if data == "idkmrc" or data == "tydiqaid":
                                url_arr.append(f"{prefix}{msc}-{data}-TQ{tq}-TS4-MS{msi}-VA{var}-TH{th}{suffix_df}")
                            elif data == "squadid":
                                url_arr.append(f"{prefix}{msc}-{data}-TQ{tq}-TS4-MS{msi}-VA{var}-TH{th}{suffix_squadid_df}")
    
    url_arr = list(OrderedDict.fromkeys(url_arr))
    print("Amount of url:", len(url_arr))
    return url_arr

In [10]:
def extract_values_nli_validator(url, data):

    msc_list = f"indonli_mnli_{data}-nli|indonli_mnli|indonli"
    pattern = re.compile(r'.*FilteringNLI-({})-{}-TQ(\d+)-TS(\d+)-MS([\d]+)-VA([\d]+)-TH([\d.]+).*'.format(msc_list, data))
    match = pattern.match(url)
    
    if match:
        msc = match.group(1)
        TQ = int(match.group(2))
        TS = int(match.group(3))
        MSI = int(match.group(4))
        VA = int(match.group(5))
        
        TH = float(match.group(6))
        if TH == '0.00':
            TH = '0.0'
        elif TH == '0.50':
            TH = '0.5'

        return msc, data, TQ, TS, MSI, VA, TH
    
    else:
        return None

In [11]:
def create_table_nli_validator(url_list, url_list_df, data):
    
    data_list = []
    for url in tqdm(url_list, desc="Processing first URLs", unit="URL"):
        extracted_values = extract_values_nli_validator(url, data)
        data_list.append(extracted_values)

    df = pd.DataFrame(data_list, columns=['MSC', 'data', 'TQ', 'TS', 'MSI', 'VA', 'TH'])
    
    for idx, url_df in enumerate(tqdm(url_list_df, desc="Processing second URLs", unit="URL")):
        extracted_values = to_right_or_wrong(url_df)
        df.loc[idx, ['WR_R', 'WR_W']] = extracted_values
    
    df[['WR_R', 'WR_W']] = df[['WR_R', 'WR_W']].astype(int)
    
    df.loc[0, 'MSC'] = "Baseline"
    df.loc[0, 'TQ'] = "BL"
    df.loc[0, 'TS'] = "BL"
    df.loc[0, 'MSI'] = "BL"
    df.loc[0, 'VA'] = "BL"
    df.loc[0, 'TH'] = "BL"
    
    return df

In [12]:
to_right_or_wrong(url)

(5, 31)

In [13]:
data = "idkmrc"
variation = "all"
url_list = take_all_url_nli_validator(data)
url_list_df = take_all_url_nli_validator(data, df=True)
table_all = pd.DataFrame(create_table_nli_validator(url_list, url_list_df, data))
#table_all.to_excel(f'{data}-nli-validator.xlsx', index=False)
table_all

Amount of url: 49
Amount of url: 49


Processing second URLs: 100%|██████████████████████████████████████████████████████████| 49/49 [00:29<00:00,  1.64URL/s]


,MSC,data,TQ,TS,MSI,VA,TH,WR_R,WR_W
0,Baseline,idkmrc,BL,BL,BL,BL,BL,0,0
1,indonli,idkmrc,1,4,1,1,0.0,11,36
2,indonli,idkmrc,1,4,2,1,0.0,11,56
3,indonli,idkmrc,1,4,3,1,0.0,10,61
4,indonli,idkmrc,2,4,1,1,0.0,3,6
5,indonli,idkmrc,2,4,2,1,0.0,3,8
6,indonli,idkmrc,2,4,3,1,0.0,3,8
7,indonli_mnli,idkmrc,1,4,1,1,0.0,13,38
8,indonli_mnli,idkmrc,1,4,2,1,0.0,14,54
9,indonli_mnli,idkmrc,1,4,3,1,0.0,13,59
